<a href="https://colab.research.google.com/github/philsML/Classification_by_Keyword/blob/master/MLwP_Group_Projects_Keywords_Application_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Import Libraries & Mount Drive.*

In [0]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
import matplotlib.pyplot as plt
from google.colab import drive
import random
from pprint import pprint
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, RandomizedSearchCV, GridSearchCV, cross_val_score, ShuffleSplit, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils import resample

In [0]:
#Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression, BayesianRidge, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB

In [0]:
random.seed(123)

In [23]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*Load Dataset.*

In [0]:
#switched to v11 to include Jakob's changes - please re-run the preprocessing code, if colab can't find the file
data=pd.read_csv('/content/drive/My Drive/ML Python_Group Project/Table for Analysis_v11.csv',sep=";")
data.rename({"Unnamed: 0":"ID"}, axis='columns',inplace=True)
cols = data.columns.to_list()

*Setup & Fit best Models from Compiled Analysis.*

In [25]:
#class_weight1 = {'F':1.19, 'M':1}

rf = RandomForestClassifier(n_estimators = 100,
                            min_samples_split= 250,
                            min_samples_leaf = 5,
                            max_features="auto",
                            max_depth=40,
                            bootstrap=True)
rf.fit(data[data.columns[4:]],np.array(data.iloc[:,3:4]).flatten())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=40, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=250,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [26]:
RFR = RandomForestRegressor(n_estimators = 200,
                            max_depth = 50, 
                            min_samples_split = 15,
                            min_samples_leaf = 10)
RFR.fit(data[data.columns[4:]],np.array(data.iloc[:,1:2]).flatten())

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=50, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=10,
                      min_samples_split=15, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [27]:
RFC_2 = RandomForestClassifier(n_estimators = 50,
                            max_depth = 50, 
                            min_samples_split = 15,
                            min_samples_leaf = 10)
RFC_2.fit(data[data.columns[4:]],np.array(data.iloc[:,2:3]).flatten())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=15,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

*Define Function for Age Clusters.*

In [0]:
def to_age_cat(input_variable):
  if input_variable=="A":
    return("14-24")
  elif input_variable=="B":
    return("25-34")
  elif input_variable=="C":
    return("35-44")
  elif input_variable=="D":
    return("45-54")
  elif input_variable=="E":
    return("55-64")    
  else:
    return("65-99")

*Set Lists with Locations of Input- and Output-Files to work through in the Loop.*

In [0]:
input_files = ['/content/drive/My Drive/ML Python_Group Project/apply_500k.csv',
               '/content/drive/My Drive/ML Python_Group Project/apply_1-0M.csv',
               '/content/drive/My Drive/ML Python_Group Project/apply_1-5M.csv',
               '/content/drive/My Drive/ML Python_Group Project/apply_2-0M.csv',
               '/content/drive/My Drive/ML Python_Group Project/apply_2-5M.csv',
               '/content/drive/My Drive/ML Python_Group Project/apply_3-0M.csv']
output_files = ['/content/drive/My Drive/ML Python_Group Project/scored_500k.csv',
               '/content/drive/My Drive/ML Python_Group Project/scored_1-0M.csv',
               '/content/drive/My Drive/ML Python_Group Project/scored_1-5M.csv',
               '/content/drive/My Drive/ML Python_Group Project/scored_2-0M.csv',
               '/content/drive/My Drive/ML Python_Group Project/scored_2-5M.csv',
               '/content/drive/My Drive/ML Python_Group Project/scored_3-0M.csv']

*Application-Loop, where for every combination of input and output (each with 500k observations) the predictions are made, combined and saved.*

In [35]:
counter=0
for inputs,outputs in zip(input_files,output_files):
  
  apply = pd.read_csv(inputs,sep=";")
  apply.drop("Unnamed: 0.1",axis="columns",inplace=True)
  apply.rename({"Unnamed: 0":"ID"}, axis='columns',inplace=True)

  pred_sex = rf.predict_proba(apply[apply.columns[1:]])
  pred_age_reg = RFR.predict(apply[apply.columns[1:]])
  pred_age_reg_bins = pd.cut(pred_age_reg, bins=[0, 24, 34, 44, 54, 64, 99])
  pred_age_cat_bins = RFC_2.predict(apply[apply.columns[1:]])

  prediction_result = pd.DataFrame(pred_age_cat_bins,columns=["Predicted_Age_Clusters_Cat"],index=apply["ID"])
  prediction_result["Predicted_Age_Clusters_Reg"]=pred_age_reg_bins
  prediction_result["Predicted_Sex"]=pred_sex[:,1]

  prediction_result["Predicted_Age_Clusters_Cat"] = prediction_result.Predicted_Age_Clusters_Cat.apply(lambda x: to_age_cat(x))

  prediction_result.to_csv(outputs,sep=";")
  
  #del(apply,prediction_result,pred_sex,pred_age_reg,pred_age_reg_bins,pred_age_cat_bins)

  counter=counter+1
  print(counter,"/6 done")

1 /6 done
2 /6 done
3 /6 done
4 /6 done
5 /6 done
6 /6 done


*Load the saved files from the application loop and concatenate them.*

In [0]:
output_1 = pd.read_csv('/content/drive/My Drive/ML Python_Group Project/scored_500k.csv',sep=";")
output_2 = pd.read_csv('/content/drive/My Drive/ML Python_Group Project/scored_1-0M.csv',sep=";")
output_3 = pd.read_csv('/content/drive/My Drive/ML Python_Group Project/scored_1-5M.csv',sep=";")
output_4 = pd.read_csv('/content/drive/My Drive/ML Python_Group Project/scored_2-0M.csv',sep=";")
output_5 = pd.read_csv('/content/drive/My Drive/ML Python_Group Project/scored_2-5M.csv',sep=";")
output_6 = pd.read_csv('/content/drive/My Drive/ML Python_Group Project/scored_3-0M.csv',sep=";")

output_list=[output_1,output_2,output_3,output_4,output_5,output_6]

In [0]:
result_table = pd.concat(output_list)

*Reset threshold slightly higher due to model's oversensitivity to "M", thus reaching a distribution of 60/40 that resembles the underlying distribution in the population (55/45) much closer.*

In [0]:
k = 0.5434
pred_sex = []
for i in result_table.Predicted_Sex:
  if i > k:
    pred_sex.append("M")
  else:
    pred_sex.append("F")
result_table["Predicted_Sex"] = pred_sex

*Drop the Age Clusters from the Classification as the model-output has small disadvantages in comparison to the Regression-output.*

In [0]:
result_table.drop("Predicted_Age_Clusters_Cat",axis=1,inplace=True)

In [103]:
result_table.tail(10)

,ID,Predicted_Age_Clusters_Reg,Predicted_Sex
248733,3111772,"(34, 44]",F
248734,3111773,"(34, 44]",M
248735,3111774,"(34, 44]",M
248736,3111775,"(44, 54]",M
248737,3111776,"(34, 44]",F
248738,3111777,"(34, 44]",M
248739,3111778,"(44, 54]",F
248740,3111779,"(34, 44]",F
248741,3111780,"(44, 54]",F
248742,3111781,"(44, 54]",M


*Save Result Table to csv for Hand-In.*

In [0]:
result_table.to_csv('/content/drive/My Drive/ML Python_Group Project/pred_results_v2.csv',sep=";")